<a href="https://colab.research.google.com/github/hamagami/ad2025/blob/main/%E6%AD%A3%E8%A6%8F%E5%88%86%E5%B8%83%E3%81%8B%E3%82%89%E3%81%AE%E7%95%B0%E5%B8%B8%E6%A4%9C%E7%9F%A5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1次元データの標準化(Zスコア)による異常検知
1次元の変数が正規分布になっていることを仮定した基本的な異常検知
1. 1次元の正規分布（相当）から学習用の「正常」データを作成
2. テスト用に「正常＋異常」を混ぜたデータを作成
3. 学習データから平均・標準偏差を推定
4. Zスコアを計算し、標準正規のしきい値で異常判定（両側）
5. 可視化（元の値のヒスト＋PDF、Zスコアのヒスト＋PDF）

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm

In [2]:
# 再現性があるようにランダムシードを固定
np.random.seed(0)

In [9]:
# ===== 1) 学習データ（正常）と 2) テストデータ（正常＋異常） =====
mu = 2.0          # 真の平均（説明用）
sigma = 1.5       # 真の標準偏差（説明用）

#　同じ平均、分散で学習データと、テストデータの一部をつくる
train = np.random.normal(mu, sigma, size=400)        # 学習：正常のみ
test_norm = np.random.normal(mu, sigma, size=200)    # テスト：正常

# テストデータに加える異常値は、平均をずらしてつくる
mu_out = mu + 6.0
sigma_out = 1.0
test_out = np.random.normal(mu_out, sigma_out, size=20)
#テストデータに加えておく
test = np.hstack([test_norm, test_out])
labels = np.hstack([np.zeros(len(test_norm), dtype=int), np.ones(len(test_out), dtype=int)])  # 0=正常,1=異常(真)

In [ ]:
# 生成したデータを可視化
# 学習データに対して、テストデータは少し右にずれたデータを加えてある。右側に外れ値がでている
plt.figure(figsize=(8, 4))
plt.hist(train, bins=30, alpha=0.6, label="Train data (normal)")
plt.hist(test, bins=30, alpha=0.6, label="Test data (normal + outlier)")
plt.title("Histograms of Train and Test Data")
plt.xlabel("Value")
plt.ylabel("Frequency")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# ===== 3) 学習データから平均・標準偏差を推定 =====
mu_hat = train.mean()
sigma_hat = train.std(ddof=1)  # 不偏標準偏差
# 理論値は平均2, 分散1.5
print(f"mean_hat={mu_hat:.3f}, std_hat={sigma_hat:.3f}")

In [15]:
# ===== 4) Zスコアの計算＆しきい値で判定（両側） =====
# Zスコアは、分布を標準化（分布を平均0, 分散1）にしたときの横軸の値
# Zスコアにして扱うことで、平均・分散にかかわらない確率的な判定が可能になる
def zscore(x, mu, s):
    return (x - mu) / s
# テストデータの各値に対して（学習データの分布を基準にした）Zスコアを求める
Z = zscore(test, mu_hat, sigma_hat)

In [ ]:
alpha = 0.99  # 「正常」とみなす両側の信頼水準（例: 0.99 → 有意水準1%）つまり平均から遠く離れた1% のデータは異常とみなすという意味
# 両側判定の場合は上側0.5%, 下側0.5% をはずれと定義する。片側判定の場合はどちらか1%をはずれと定義する。
# ここでは両側判定を行っているので、上側のカットオフは　(1+alpha)/2  (つまり (1+0.99)/2 = 0.995)
# norm.ppf はカットオフ値をzスコアに変換する関数。両側検定なので、下側のｚスコアは符号を判定させた値であることに注意
z_thr = norm.ppf((1 + alpha) / 2)
# つまり、zスコアの絶対値に対して判定することで、両側の判定になる
pred = (np.abs(Z) > z_thr).astype(int)  # |Z| がしきい値を超えたら「異常」

# 真値と判定値の一致度を計算
tp = ((pred==1) & (labels==1)).sum() # true positive (異常を異常と正しく判定した数)
fp = ((pred==1) & (labels==0)).sum() # false positive (正常を異常と誤って判定した数)
fn = ((pred==0) & (labels==1)).sum() # false negative (異常を正常と誤って判定した数)
precision = tp / (tp+fp) if (tp+fp)>0 else 0.0
recall    = tp / (tp+fn) if (tp+fn)>0 else 0.0

print(f"mean_hat={mu_hat:.3f}, std_hat={sigma_hat:.3f}")
print(f"two-sided z-threshold (alpha={alpha}): ±{z_thr:.3f}")
print(f"TP={tp}, FP={fp}, FN={fn}")
print(f"precision={precision:.3f}, recall={recall:.3f}")

In [ ]:
# ===== 5) 可視化 =====
# (a) 元の値のヒストグラム + 推定正規PDF + しきい値（x空間の線）
x_grid = np.linspace(min(test.min(), train.min()) - 3*sigma_hat,
                     max(test.max(), train.max()) + 3*sigma_hat, 500)
pdf = norm.pdf(x_grid, loc=mu_hat, scale=sigma_hat)

x_thr_low  = mu_hat - z_thr * sigma_hat
x_thr_high = mu_hat + z_thr * sigma_hat

plt.figure(figsize=(7,4))
plt.hist(test[labels==0], bins=30, density=True, alpha=0.6, label="Test normals")
plt.hist(test[labels==1], bins=15, density=True, alpha=0.6, label="Test outliers")
plt.plot(x_grid, pdf, label="Fitted Normal PDF")
plt.axvline(x_thr_low,  linestyle='--', label="lower threshold")
plt.axvline(x_thr_high, linestyle='--', label="upper threshold")
plt.title("Original values: histogram + fitted normal PDF")
plt.xlabel("x"); plt.ylabel("density")
plt.legend(); plt.grid(True)
plt.show()

# (b) Zスコアのヒストグラム + 標準正規PDF + しきい値（±z）
z_grid = np.linspace(-5, 5, 500)
std_pdf = norm.pdf(z_grid)  # 標準正規 N(0,1)

plt.figure(figsize=(7,4))
plt.hist(Z[labels==0], bins=30, density=True, alpha=0.6, label="Z of normals")
plt.hist(Z[labels==1], bins=15, density=True, alpha=0.6, label="Z of outliers")
plt.plot(z_grid, std_pdf, label="Standard Normal PDF")
plt.axvline(-z_thr, linestyle='--', label=f"-z_thr")
plt.axvline( z_thr, linestyle='--', label=f"+z_thr")
plt.title("Z-scores: histogram + standard normal PDF")
plt.xlabel("Z"); plt.ylabel("density")
plt.legend(); plt.grid(True)
plt.show()


データの分布に対し、右側にはみ出た分布が異常値として検出されていることがわかる